In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from datetime import datetime,date
from sqlalchemy import create_engine

# File to Load
CESDF=pd.read_pickle('resources/CES2020.pkl',compression='gzip')

# Output File (CSV)
output_data_file = "Resources/Industry_Data.csv"

# Preview of the DataFrame
CESDF.head()

,series_id,year,value,industry_code_x,Month,supersector_code,data_type_code,series_title,supersector_name,data_type_text,short_title
0,CES0000000001,2015,140568.0,00000000,1,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
1,CES0000000001,2015,140839.0,00000000,2,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
2,CES0000000001,2015,140910.0,00000000,3,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
3,CES0000000001,2015,141194.0,00000000,4,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
4,CES0000000001,2015,141525.0,00000000,5,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees


In [3]:
# Check for correct data types
CESDF.dtypes

series_id            object
year                  int64
value               float64
industry_code_x      object
Month                 int32
supersector_code     object
data_type_code       object
series_title         object
supersector_name     object
data_type_text       object
short_title          object
dtype: object

In [4]:
# Check column names for spaces
CESDF.columns

Index(['series_id', 'year', 'value', 'industry_code_x', 'Month',
       'supersector_code', 'data_type_code', 'series_title',
       'supersector_name', 'data_type_text', 'short_title'],
      dtype='object')

In [5]:
# Identify incomplete rows
CESDF.count()

series_id           4020
year                4020
value               4020
industry_code_x     4020
Month               4020
supersector_code    4020
data_type_code      4020
series_title        4020
supersector_name    4020
data_type_text      4020
short_title         4020
dtype: int64

In [7]:
# Check series values
CESDF['series_id'].value_counts()

CES6561000001    65
CES7000000001    65
CES4300000010    65
CES4200000010    65
CES8000000001    65
                 ..
CES9092000010    64
CES6056000010    64
CES9093000010    64
CES7071000010    64
CES9091000010    64
Name: series_id, Length: 62, dtype: int64

In [8]:
# Isolate data to series data needed  - Industry level employment data
series = ["CES0000000001","CES0500000001","CES0600000001","CES0700000001","CES0800000001",
"CES1000000001","CES2000000001","CES3000000001","CES3100000001","CES3200000001",
"CES4000000001","CES4200000001","CES4300000001","CES5000000001","CES5500000001",
"CES5552000001","CES5553000001","CES6000000001","CES6054000001","CES6055000001",
"CES6056000001","CES6500000001","CES6561000001","CES6562000001","CES7000000001",
"CES7071000001","CES7072000001","CES8000000001","CES9000000001","CES9091000001",
"CES9092000001","CES9093000001"]

condition = CESDF['series_id'].isin(series)
CESSeriesDF = CESDF[condition]
CESSeriesDF.head(5)

,series_id,year,value,industry_code_x,Month,supersector_code,data_type_code,series_title,supersector_name,data_type_text,short_title
0,CES0000000001,2015,140568.0,00000000,1,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
1,CES0000000001,2015,140839.0,00000000,2,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
2,CES0000000001,2015,140910.0,00000000,3,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
3,CES0000000001,2015,141194.0,00000000,4,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees
4,CES0000000001,2015,141525.0,00000000,5,00,01,"All employees, thousands, total nonfarm, seaso...",Total nonfarm,"ALL EMPLOYEES, THOUSANDS",All employees


In [9]:
# Get rows and columns count
CESSeriesDF.shape

(2080, 11)

In [10]:
# Choose columns and create a date column with year and month
df = CESSeriesDF[['year','Month','value','short_title']]
df['date']=df['year'].astype(str) + "-"+ df['Month'].astype(str) 
df.head()

C:\Users\danjr\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,year,Month,value,short_title,date
0,2015,1,140568.0,All employees,2015-1
1,2015,2,140839.0,All employees,2015-2
2,2015,3,140910.0,All employees,2015-3
3,2015,4,141194.0,All employees,2015-4
4,2015,5,141525.0,All employees,2015-5


In [19]:
# Rename short_title categories
df['industry']=df['short_title'].replace({
        'All employees':'All Employees',
        'All employees accommodation and food services':'Accomm. & Food Services',
        'All employees administrative and waste services': 'Admin. & Waste Services',
        'All employees arts entertainment and recreation': 'Arts Ent. & Recreation',
        'All employees construction':'Construction',
        'All employees durable goods': 'Durable Goods',
        'All employees education and health services': 'Edu. & Health Services',
        'All employees educational services':'Educational Services',
        'All employees federal':'Federal',
        'All employees finance and insurance':'Finance & Insurance',
        'All employees financial activities':'Financial Activities',
        'All employees goods-producing':'Goods Producing',
        'All employees government':'Government',
        'All employees health care and social assistance':'Healthcare & Social Assistance',
        'All employees information':'Information',
        'All employees leisure and hospitality':'Leisure & Hospitality',
        'All employees local government':'Local Government',
        'All employees management of companies and enterprises':'Mgmt Companies & Enterprises',
        'All employees manufacturing':'Manufacturing',
        'All employees mining and logging':'Mining & Logging',
        'All employees nondurable goods':'Nondurable Goods',
        'All employees other services':'Other Services',
        'All employees private':'Private',
        'All employees private service-providing':'Private Service Providing',
        'All employees professional and business services':'Professional & Business Services',
        'All employees professional and technical services':'Professional & Technical Services',
        'All employees real estate and rental and leasing':'Real Estate & Rental',
        'All employees retail trade':'Retail Trade',
        'All employees service-providing':'Service Providing',
        'All employees state government':'State Government',
        'All employees trade transportation and utilities':'Trade Transportation & Utilities',
        'All employees transportation and warehousing':'Transportation & Warehousing',
    })
df.head()    

C:\Users\danjr\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,year,Month,value,short_title,date,industry
0,2015,1,140568.0,All employees,2015-1,All Employees
1,2015,2,140839.0,All employees,2015-2,All Employees
2,2015,3,140910.0,All employees,2015-3,All Employees
3,2015,4,141194.0,All employees,2015-4,All Employees
4,2015,5,141525.0,All employees,2015-5,All Employees


In [20]:
# Rename columns and drop short_tite
df=df.rename(
    columns = {
        'Month':'month',
        'value':'number_of_persons'
        })

final_df = df[['date','year','month','industry','number_of_persons']]
final_df.head()

,date,year,month,industry,number_of_persons
0,2015-1,2015,1,All Employees,140568.0
1,2015-2,2015,2,All Employees,140839.0
2,2015-3,2015,3,All Employees,140910.0
3,2015-4,2015,4,All Employees,141194.0
4,2015-5,2015,5,All Employees,141525.0


In [21]:
# Export clean data to CSV for analysis
final_df.to_csv(output_data_file, index = False)

In [22]:
# Push to SQLite
engine = create_engine('sqlite:///Resources/Industry_Data.db', echo=False)
final_df.to_sql('industry_data',con=engine,if_exists='replace')